# Домашнее задание № 2. Мешок слов

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

from IPython.display import Image
from IPython.core.display import HTML 

In [2]:
import numpy as np

In [3]:
from razdel import tokenize as razdel_tokenize

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор (любой) с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [4]:
data = pd.read_csv('labeled.csv')

In [5]:
train, test = train_test_split(data, test_size=0.1, shuffle=True)

In [6]:
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [7]:
vectorizer_default = TfidfVectorizer(min_df=10, max_df=0.3)
X_default = vectorizer_default.fit_transform(train.comment)
X_test_default = vectorizer_default.transform(test.comment) 

In [8]:
y_default = train.toxic.values
y_test_default = test.toxic.values

In [8]:
clf_default = KNeighborsClassifier(n_neighbors=10, metric='cosine')
clf_default.fit(X_default, y_default)
preds_default = clf_default.predict(X_test_default)

print(classification_report(y_test_default, preds_default))

              precision    recall  f1-score   support

         0.0       0.82      0.86      0.84       954
         1.0       0.70      0.64      0.67       488

    accuracy                           0.78      1442
   macro avg       0.76      0.75      0.75      1442
weighted avg       0.78      0.78      0.78      1442



In [12]:
def r_tokenizer(text):
    tokens = [token.text for token in list(razdel_tokenize(text))]
    return tokens

In [10]:
vectorizer_custom = TfidfVectorizer(tokenizer=r_tokenizer, min_df=10, max_df=0.3)
X_custom = vectorizer_custom.fit_transform(train.comment)
X_test_custom = vectorizer_custom.transform(test.comment) 

c:\Users\Home\scoop\apps\python\current\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [11]:
y_custom = train.toxic.values
y_test_custom = test.toxic.values

In [15]:
clf_custom = KNeighborsClassifier(n_neighbors=10, metric='cosine')
clf_custom.fit(X_custom, y_custom)
preds_custom = clf_custom.predict(X_test_custom)

print(classification_report(y_test_custom, preds_custom))

              precision    recall  f1-score   support

         0.0       0.82      0.83      0.83       954
         1.0       0.67      0.65      0.66       488

    accuracy                           0.77      1442
   macro avg       0.74      0.74      0.74      1442
weighted avg       0.77      0.77      0.77      1442



### Наблюдения
Токенайзер из razdel показал хорошие результаты, но все таки они чуть хуже дефолтного токенайзера.

## Задание 2 (3 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к моделям:   
а) один классификатор должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров (можно ставить разные параметры tfidfvectorizer и countvectorizer)  
в) у классификатора должно быть задано вручную как минимум 2 параметра (по возможности)  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

*random_seed не считается за параметр

In [9]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Home\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
stops = stopwords.words('russian')

In [25]:
vectorizer = TfidfVectorizer(min_df=1, max_df=0.3, stop_words=stops, tokenizer = r_tokenizer, lowercase=True)
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment) 

y = train.toxic.values
y_test = test.toxic.values

clf = KNeighborsClassifier(n_neighbors=10, n_jobs = 2, metric = 'cosine')
clf.fit(X, y)
preds = clf.predict(X_test)

print(classification_report(y_test, preds))

c:\Users\Home\scoop\apps\python\current\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


              precision    recall  f1-score   support

         0.0       0.79      0.97      0.87       944
         1.0       0.90      0.51      0.65       498

    accuracy                           0.81      1442
   macro avg       0.84      0.74      0.76      1442
weighted avg       0.83      0.81      0.79      1442



In [29]:
percents_top10 = list(np.sort(clf.predict_proba(X_test)[:, 1])[::-1][:10])
indices_top10  = list(np.argsort(clf.predict_proba(X_test)[:, 1])[::-1][:10])
top10_sents = [vectorizer.inverse_transform(X_test[i]) for i in indices_top10]
top10_sents

[[array(['хохол'], dtype='<U67')],
 [array(['татарин', 'пидарасы', 'мимо', 'астраханский'], dtype='<U67')],
 [array(['чмо', 'ха', 'лох', '!'], dtype='<U67')],
 [array(['хохлы'], dtype='<U67')],
 [array(['хохлов', 'забыл'], dtype='<U67')],
 [array(['свидетель', 'петух'], dtype='<U67')],
 [array(['прокладки', 'ебать', 'бомбит'], dtype='<U67')],
 [array(['чурка', 'тащемта', 'татарин', 'попали', 'леваки', 'дурак', 'геи'],
        dtype='<U67')],
 [array(['шлюха', 'хуесос', 'пидорас', 'оп', 'мать', 'конченный', '-'],
        dtype='<U67')],
 [array(['маски', 'лицо', 'жмет'], dtype='<U67')]]

In [16]:
vectorizer = CountVectorizer(min_df=1, max_df=0.3, stop_words=stops, tokenizer = r_tokenizer, lowercase=True)
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment) 

y = train.toxic.values
y_test = test.toxic.values

clf = LogisticRegression(C=0.1, penalty='elasticnet', class_weight='balanced', max_iter=100, solver='saga', l1_ratio = 0.5)
clf.fit(X, y)
preds = clf.predict(X_test)

print(classification_report(y_test, preds))

c:\Users\Home\scoop\apps\python\current\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


              precision    recall  f1-score   support

         0.0       0.87      0.71      0.78       953
         1.0       0.59      0.80      0.68       489

    accuracy                           0.74      1442
   macro avg       0.73      0.75      0.73      1442
weighted avg       0.78      0.74      0.75      1442



c:\Users\Home\scoop\apps\python\current\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [17]:
percents_top10 = list(np.sort(clf.predict_proba(X_test)[:, 1])[::-1][:10])
indices_top10  = list(np.argsort(clf.predict_proba(X_test)[:, 1])[::-1][:10])
top10_sents = [vectorizer.inverse_transform(X_test[i]) for i in indices_top10]
top10_sents

[[array(['!', '-', ':', '?', 'баный', 'белых', 'бля', 'блядь', 'будем',
         'вводит', 'вегетарианство', 'верит', 'вещи', 'вмешивается',
         'вокруг', 'всем', 'вспомни', 'всё', 'выстрелить', 'говорю',
         'головой', 'голубей', 'дальше', 'делал', 'дело', 'должен', 'друг',
         'друга', 'дураком', 'ебучие', 'ем', 'ещё', 'жалко', 'жгли',
         'животных', 'живьём', 'заботится', 'заботиться', 'заставляет',
         'итог', 'картошка', 'картошку', 'которых', 'кошек', 'любить',
         'люблю', 'люди', 'медленно', 'могу', 'можешь', 'мышей', 'мяса',
         'мясо', 'начинают', 'неправильно', 'неё', 'объясняет', 'огонь',
         'организм', 'остальные', 'отвечает', 'перевести', 'пизди',
         'пиздёж', 'поддержать', 'подростковых', 'показывает', 'показывая',
         'полному', 'полным', 'попытку', 'порыв', 'послать', 'поступает',
         'почему', 'правильно', 'представляю', 'просто', 'равно', 'разные',
         'самое', 'самые', 'свинью', 'своим', 'свой', 'сказал'

## Задание 3 (4 балла - 1 балл за каждый классификатор)

Для классификаторов Logistic Regression, Decision Trees, Naive Bayes, RandomForest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию. 
Также как и в предыдущем задании у классификаторов должно быть задано вручную как минимум 2 параметра (по возможности, f1 мера каждого из классификаторов должна быть минимум 0.75

In [22]:
# LogReg
vectorizer = CountVectorizer(stop_words=stops)

X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment)

y = train.toxic.values
y_test = test.toxic.values

clf = LogisticRegression(C=0.1, penalty='elasticnet', class_weight='balanced', max_iter=100, solver='saga', l1_ratio = 0.5)
clf.fit(X, y)
preds = clf.predict(X_test)

print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.91      0.70      0.79       953
         1.0       0.60      0.87      0.71       489

    accuracy                           0.76      1442
   macro avg       0.75      0.78      0.75      1442
weighted avg       0.80      0.76      0.76      1442



c:\Users\Home\scoop\apps\python\current\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [23]:
top_index = []

for i in np.sort(clf.coef_[0])[::-1][:12]:
  top_index.append(list(clf.coef_[0]).index(i))
vectorizer.get_feature_names_out()[top_index][2:7]

array(['тебе', 'нахуй', 'блядь', 'блять', 'пиздец'], dtype=object)

In [24]:
clf = RandomForestClassifier(n_estimators=300, max_depth=500)
clf.fit(X, y)
preds = clf.predict(X_test)

print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.77      0.97      0.86       953
         1.0       0.90      0.42      0.57       489

    accuracy                           0.79      1442
   macro avg       0.83      0.70      0.71      1442
weighted avg       0.81      0.79      0.76      1442



In [25]:
top_index = []

for i in np.sort(clf.feature_importances_)[::-1][:20]:
  top_index.append(list(clf.feature_importances_).index(i))
vectorizer.get_feature_names_out()[top_index][:5]

array(['хохлов', 'хохлы', 'тебе', 'нахуй', 'очень'], dtype=object)